In [1]:
from glob import glob
import json
import soundfile as sf
import IPython.display as ipd
import mp

In [2]:
!pip3 install soundfile

In [3]:
filtered = glob('tts-filtered/*.json')
len(filtered)

1645455

In [4]:
!rm -rf tts-filtered-chunk tts-filtered-chunk-rows tts-filtered-chunk-rows-audio
!mkdir tts-filtered-chunk tts-filtered-chunk-rows tts-filtered-chunk-rows-audio

In [5]:
import random
import soundfile as sf
from tqdm import tqdm

def loop(files):
    files, _ = files
    for f in tqdm(files):
        with open(f) as fopen:
            d = json.load(fopen)
        index = d['index']
        speaker = d['speaker']
        
        filename_done = f'tts-filtered-chunk/{index}.json'
        
        c = d['alignment']
        pitch = d['averaged_pitch']
        speed = d['distances']
        segments = []
        segments_pitch = []
        segments_speed = []
        temp = [c[0]]
        temp_pitch = [pitch[0]]
        temp_speed = [speed[0]]
        last_t = c[0]['end']
        total = 0
        for i in range(1, len(c), 1):
            c_ = c[i]
            if (c_['start'] - last_t) >= 0.17:
                total += 1
                t = ' '.join([t_['text'] for t_ in temp])
                if len(t) > 2 and t != 'uhm,':
                    segments.append(temp)
                    segments_pitch.append(temp_pitch)
                    segments_speed.append(temp_speed)
                temp = []
                temp_pitch = []
                temp_speed = []

            last_t = c_['end']
            temp.append(c_)
            temp_pitch.append(pitch[i])
            temp_speed.append(speed[i])


        if len(temp):
            total += 1
            t = ' '.join([t_['text'] for t_ in temp])
            if len(t) > 2:
                segments.append(temp)
                segments_pitch.append(temp_pitch)
                segments_speed.append(temp_speed)
                
        if total > 1:
            y, sr = sf.read(d['filename_audio'])
            for k in range(len(segments)):
                filename_audio = f'tts-filtered-chunk-rows-audio/{index}-{k}.mp3'
                filename = f'tts-filtered-chunk-rows/{index}-{k}.json'
                start = segments[k][0]['start']
                end = segments[k][-1]['end'] + 0.15
                y_ = y[int(start * sr): int(end * sr)]
                sf.write(filename_audio, y_, sr)
                
                d_ = {
                    'filename_audio': filename_audio,
                    'speaker': speaker,
                    'alignment': segments[k],
                    'averaged_pitch': segments_pitch[k],
                    'distances': segments_speed[k]
                }
                with open(filename, 'w') as fopen:
                    json.dump(d_, fopen)
        
        with open(filename_done, 'w') as fopen:
            json.dump('done', fopen)

In [6]:
loop((filtered[:10], 0))

100%|██████████| 10/10 [00:01<00:00,  8.46it/s]


In [7]:
mp.multiprocessing(filtered, loop, cores = 200)

 21%|██        | 1689/8227 [07:58<21:13,  5.13it/s]  IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

 39%|███▉      | 3188/8227 [15:28<27:39,  3.04it/s]  IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

 48%|████▊     | 3950/8227 [19:35<20:01,  3.56it/s]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_lim

In [11]:
!du -hs tts-filtered-chunk-rows-audio

112G	tts-filtered-chunk-rows-audio
